In [1]:
# create bucket - lpkadvworks
!gsutil mb gs://lpkadvworks

In [2]:
!gsutil ls 

Creating gs://lpkadvworks/...
ServiceException: 409 A Cloud Storage bucket named 'lpkadvworks' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [3]:
# copy data directory to GCS bucket
!gsutil cp -r ../data gs://lpkadvworks

gs://lpkadvworks/
gs://lpkretail/


In [4]:
# validate for files in bucket
!gsutil ls -r gs://lpkadvworks

Copying file://..\data\AdventureWorks_Calendar.csv [Content-Type=application/vnd.ms-excel]...
/ [0 files][    0.0 B/  9.7 KiB]                                                
/ [1 files][  9.7 KiB/  9.7 KiB]                                                
Copying file://..\data\AdventureWorks_Customers.csv [Content-Type=application/vnd.ms-excel]...
/ [1 files][  9.7 KiB/  1.9 MiB]                                                
-
- [2 files][  1.9 MiB/  1.9 MiB]                                                
Copying file://..\data\AdventureWorks_Products.csv [Content-Type=application/vnd.ms-excel]...
- [2 files][  1.9 MiB/  1.9 MiB]                                                
\
\ [3 files][  1.9 MiB/  1.9 MiB]                                                
Copying file://..\data\AdventureWorks_Product_Categories.csv [Content-Type=application/vnd.ms-excel]...
\ [3 files][  1.9 MiB/  1.9 MiB]                                                
\ [4 files][  1.9 MiB/  1.9 MiB]          

In [5]:
# import storage from google cloud
from google.cloud import storage 

In [6]:
# instantiate gsclient
gsclient = storage.Client()

gs://lpkadvworks/data/:
gs://lpkadvworks/data/AdventureWorks_Calendar.csv
gs://lpkadvworks/data/AdventureWorks_Customers.csv
gs://lpkadvworks/data/AdventureWorks_Product_Categories.csv
gs://lpkadvworks/data/AdventureWorks_Product_Subcategories.csv
gs://lpkadvworks/data/AdventureWorks_Products.csv
gs://lpkadvworks/data/AdventureWorks_Returns.csv
gs://lpkadvworks/data/AdventureWorks_Territories.csv

gs://lpkadvworks/data/AW_Sales/:
gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2015.csv
gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2015.snappy.parquet
gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2016.csv
gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2017.csv


In [7]:
# list buckets
all_buckets = list(gsclient.list_buckets())
all_buckets

[<Bucket: lpkadvworks>, <Bucket: lpkretail>]

In [8]:
# check for all buckets
nums = 0
for bucket in all_buckets:
    nums +=1
    print(f'bucket {nums}: {bucket.name}')
print(f'There are {nums} buckets in my cloud storage!')

bucket 1: lpkadvworks
bucket 2: lpkretail
There are 2 buckets in my cloud storage!


In [9]:
# look at objects in lpkadvworks bucket
counter = 0
all_files_lpkadvworks = list(gsclient.list_blobs('lpkadvworks'))
print(f'There are {len(all_files_lpkadvworks)} main files/folders in lpkadvworks bucket.')
print('They are:')
for file_folder in all_files_lpkadvworks:
    counter += 1
    print(f'{counter}: {file_folder.name}')

There are 11 main files/folders in lpkadvworks bucket.
They are:
1: data/AW_Sales/AdventureWorks_Sales_2015.csv
2: data/AW_Sales/AdventureWorks_Sales_2015.snappy.parquet
3: data/AW_Sales/AdventureWorks_Sales_2016.csv
4: data/AW_Sales/AdventureWorks_Sales_2017.csv
5: data/AdventureWorks_Calendar.csv
6: data/AdventureWorks_Customers.csv
7: data/AdventureWorks_Product_Categories.csv
8: data/AdventureWorks_Product_Subcategories.csv
9: data/AdventureWorks_Products.csv
10: data/AdventureWorks_Returns.csv
11: data/AdventureWorks_Territories.csv


In [23]:
import pandas as pd
df_sales_15 = pd.read_csv('gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2015.csv')
df_sales_15.tail()

,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
2625,12/31/2015,11/29/2002,SO48728,354,13111,9,1,1
2626,12/31/2015,11/14/2002,SO48729,324,26563,9,1,1
2627,12/31/2015,12/2/2002,SO48724,340,20722,8,1,1
2628,12/31/2015,10/9/2002,SO48723,369,14944,7,1,1
2629,12/31/2015,11/22/2002,SO48726,383,24915,9,1,1


In [11]:
df_sales_15.to_parquet('gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2015.snappy.parquet', index=False)

In [12]:
df_parquet = pd.read_parquet('gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2015.snappy.parquet')
df_parquet.head()

,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
0,1/1/2015,9/21/2001,SO45080,332,14657,1,1,1
1,1/1/2015,12/5/2001,SO45079,312,29255,4,1,1
2,1/1/2015,10/29/2001,SO45082,350,11455,9,1,1
3,1/1/2015,11/16/2001,SO45081,338,26782,6,1,1
4,1/2/2015,12/15/2001,SO45083,312,14947,10,1,1


In [13]:
gsclient.list_blobs?

Signature:
gsclient.list_blobs(
    bucket_or_name,
    max_results=None,
    page_token=None,
    prefix=None,
    delimiter=None,
    start_offset=None,
    end_offset=None,
    include_trailing_delimiter=None,
    versions=None,
    projection='noAcl',
    fields=None,
    page_size=None,
    timeout=60,
    retry=<google.api_core.retry.Retry object at 0x000001969E45C7F0>,
)
Docstring:
Return an iterator used to find blobs in the bucket.

If :attr:`user_project` is set, bills the API request to that project.

.. note::
  List prefixes (directories) in a bucket using a prefix and delimiter.
  See a [code sample](https://cloud.google.com/storage/docs/samples/storage-list-files-with-prefix#storage_list_files_with_prefix-python)
  listing objects using a prefix filter.

Args:
    bucket_or_name (Union[                 :class:`~google.cloud.storage.bucket.Bucket`,                  str,             ]):
        The bucket resource to pass or name to create.

    max_results (int):
        

In [14]:
lpkadvworks_bucket = gsclient.bucket('lpkadvworks')
blob_data = lpkadvworks_bucket.get_blob('data/AW_Sales/AdventureWorks_Sales_2015.csv')


In [15]:
blob_data.name

'data/AW_Sales/AdventureWorks_Sales_2015.csv'

In [16]:
print(blob_data.generation)

1673856529605551


In [17]:
blob_data.time_created

datetime.datetime(2023, 1, 16, 8, 8, 49, 752000, tzinfo=datetime.timezone.utc)

In [18]:
from datetime import datetime
blob_data.time_created.date()

datetime.date(2023, 1, 16)

In [19]:
print(blob_data.time_created.timetz())

08:08:49.752000+00:00
